In [2]:
import pandas as pd
import numpy as np
dato = pd.read_csv('properatti.csv')
datodf = pd.DataFrame(dato)
pd.set_option('display.float_format', lambda x: '%.2f' % x)

##### Sumarizo los nullls para tener una nocion general

In [3]:
np.sum(datodf.isnull());

#### Me centrare en las columnas 'place_name', 'price', 'currency', 'price_aprox_local_currency', 'price_aprox_usd', 'surface_total_in_m2', 'surface_covered_in_m2', 'price_usd_per_m2', 'price_per_m2'

In [4]:
mycolums = datodf[['place_name', 'price', 'currency',
       'price_aprox_local_currency', 'price_aprox_usd', 'surface_total_in_m2',
       'surface_covered_in_m2', 'price_usd_per_m2', 'price_per_m2', 'title', 'description']];

#### Tengo 20 mil casos en los que no tengo ningun dato de precio:

In [5]:
mycolums.loc[((datodf['price'].isnull()) & (datodf['price_aprox_usd'].isnull())& \
           (datodf['price_aprox_local_currency'].isnull())\
            &(datodf['price_usd_per_m2'].isnull()) & (datodf['price_per_m2'].isnull()))];

#### Creo el DF dfprecio para trabajar todo lo relacionado a precio en el

In [6]:
dfprecio = mycolums.loc[((datodf['price'].isnull()) & (datodf['price_aprox_usd'].isnull())& \
           (datodf['price_aprox_local_currency'].isnull())\
            &(datodf['price_usd_per_m2'].isnull()) & (datodf['price_per_m2'].isnull()))];

#### Verifico los casos en los que no tienen precio de ningun tipo y son planes ahorro (cuotas) Lo dropeo

In [8]:
dfprecio = dfprecio.loc[~dfprecio['title'].str.contains(r'cuotas')];
dfprecio = dfprecio.loc[~dfprecio['title'].str.contains(r'CUOTA')];

#### Extraigo precios de los titulos y descripciones. Creo 2 dataframes 


In [9]:
df_precio_desc = dfprecio['description'].str.extract(r'([$]|[Uu][Ss$][$SDsd]*)\s*(\d*)[\s* .,]*(\d*)[\s* .,](\d*)')
df_precio_title = dfprecio['title'].str.extract(r'([$]|[Uu][Ss$][$SDsd]*)\s*(\d*)[\s* .,]*(\d*)[\s* .,](\d*)')

#### Sumarizo los resultados de los grupos de captura y creo un nuevo dataframe con esos datos.

In [14]:
df_precio_title['sumat'] = df_precio_title[1] + df_precio_title[2] + df_precio_title[3]
df_precio_desc['sumad'] = df_precio_desc[1] + df_precio_desc[2] + df_precio_desc[3]

In [15]:
df_precio_title2 = df_precio_title.iloc[:,[0,4]]
df_precio_desc2 = df_precio_desc.iloc[:,[0,4]]

In [20]:
df_precio_desc2['sumad'].fillna(0, inplace = True);

In [21]:
df_precio_title2['sumat'].fillna(0, inplace = True);

In [24]:
df_precio_title2.replace("", 0);

In [33]:
df_precio_desc2['sumad'] = df_precio_desc2.sumad.str.strip();

/Users/Leandro/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [35]:
df_precio_desc2.replace("", 0, inplace=True)

/Users/Leandro/anaconda3/lib/python3.7/site-packages/pandas/core/frame.py:4042: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  method=method)


In [36]:
df_precio_desc2['sumad'].fillna(0 ,inplace=True)

/Users/Leandro/anaconda3/lib/python3.7/site-packages/pandas/core/generic.py:6130: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._update_inplace(new_data)


In [40]:
dfpredesc2 = pd.merge(df_precio_desc2, df_precio_title2, how='outer', on=df_precio_desc2.index)

In [41]:
dfpredesc2['sumad'] = pd.to_numeric(dfpredesc2['sumad'])
dfpredesc2['sumat'] = pd.to_numeric(dfpredesc2['sumat'])

In [43]:
dfpredesc2['final'] = dfpredesc2[['sumad','sumat']].max(axis=1)